In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
from pyvbmc import VBMC
import corner
from tqdm import tqdm
import random
from scipy.integrate import cumulative_trapezoid as cumtrapz

from time_vary_norm_utils import (
    up_or_down_RTs_fit_fn, cum_pro_and_reactive_time_vary_fn,
    rho_A_t_VEC_fn, up_or_down_RTs_fit_wrt_stim_fn)
from types import SimpleNamespace
import pickle

from time_vary_and_norm_simulators import psiam_tied_data_gen_wrapper_rate_norm_time_vary_fn
from tqdm.notebook import tqdm
from scipy.integrate import trapezoid as trapz
from time_vary_and_norm_simulators import phi_t_fn


In [2]:
# read out_LED.csv as dataframe
og_df = pd.read_csv('../out_LED.csv')

# chose non repeat trials - 0 or 2 or missing
df = og_df[ og_df['repeat_trial'].isin([0,2]) | og_df['repeat_trial'].isna() ]

# only session type 7
session_type = 7    
df = df[ df['session_type'].isin([session_type]) ]

# training level 16
training_level = 16
df = df[ df['training_level'].isin([training_level]) ]

df = df[  df['LED_trial'] == 0 ]

# 1 is right , -1 is left
df['choice'] = df['response_poke'].apply(lambda x: 1 if x == 3 else (-1 if x == 2 else random.choice([1, -1])))

# 1 or 0 if the choice was correct or not
df['correct'] = (df['ILD'] * df['choice']).apply(lambda x: 1 if x > 0 else 0)

# find ABL and ILD
ABL_arr = df['ABL'].unique()
ILD_arr = df['ILD'].unique()


# sort ILD arr in ascending order
ILD_arr = np.sort(ILD_arr)
ABL_arr = np.sort(ABL_arr)

print('ABL:', ABL_arr)
print('ILD:', ILD_arr)

ABL: [20 40 60]
ILD: [-16.  -8.  -4.  -2.  -1.   1.   2.   4.   8.  16.]


In [3]:
# LED off rows
df_led_off = df[ df['LED_trial'] == 0 ]
print(f'len df_led_off = {len(df_led_off)}')

# > 0 and < 1s valid rt 
df_led_off_valid = df_led_off[
    (df_led_off['timed_fix'] - df_led_off['intended_fix'] > 0) &
    (df_led_off['timed_fix'] - df_led_off['intended_fix'] < 1)
]

df_led_off_valid = df_led_off_valid[df_led_off_valid['response_poke'].isin([2,3])]

print(len(df_led_off_valid))

len df_led_off = 66226
52799


## ******* REMOVE ILD 16 **************

In [4]:
df_led_off_valid = df_led_off_valid[~df_led_off_valid['ILD'].isin([16, -16])].copy()
print(f'len of df_led_off_valid = {len(df_led_off_valid)}')

len of df_led_off_valid = 42096


In [5]:
V_A = 1.596
theta_A = 2.531
t_A_aff = -0.226
# del_go = 0.13 # TO BE FIT


# fix lambda and theta
rate_lambda = 1.935
theta_E = 3.03

In [6]:
# phi params
bump_offset = 0
# bump_width = 0.240
# dip_width = 0.038
# dip_height = 0.327

K_max = 10

# vbmc

In [7]:
is_norm = False
is_time_vary = True

In [8]:
rate_norm_l = np.nan

## loglike fn

In [9]:
def compute_loglike(row, T_0, Z_E, t_E_aff, bump_height, bump_width, dip_height, dip_width, del_go):
    
    timed_fix = row['timed_fix']
    intended_fix = row['intended_fix']
    
    
    ILD = row['ILD']
    ABL = row['ABL']
    choice = 2*row['response_poke'] - 5

    rt = timed_fix
    t_stim = intended_fix

    phi_params = {
        'h1': bump_width,
        'a1': bump_height,
        'h2': dip_width,
        'a2': dip_height,
        'b1': bump_offset
    }

    phi_params_obj = SimpleNamespace(**phi_params)
    
    trunc_factor_p_joint = cum_pro_and_reactive_time_vary_fn(
                            t_stim + 1,
                            V_A, theta_A, t_A_aff,
                            t_stim, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_E_aff,
                            phi_params_obj, rate_norm_l, 
                            is_norm, is_time_vary, K_max) \
                            - \
                            cum_pro_and_reactive_time_vary_fn(
                            t_stim,
                            V_A, theta_A, t_A_aff,
                            t_stim, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_E_aff,
                            phi_params_obj, rate_norm_l, 
                            is_norm, is_time_vary, K_max) + 1e-10
    P_joint_rt_choice = up_or_down_RTs_fit_fn(
                    rt, choice,
                    V_A, theta_A, t_A_aff,
                    t_stim, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_E_aff, del_go,
                    phi_params_obj, rate_norm_l, 
                    is_norm, is_time_vary, K_max)

    P_joint_rt_choice_trunc = max(P_joint_rt_choice / (trunc_factor_p_joint + 1e-10), 1e-10)
    return np.log(P_joint_rt_choice_trunc)
    
    


def vbmc_loglike_fn(params):
    T_0, w, t_E_aff, bump_height, bump_width, dip_height, dip_width, del_go = params
    Z_E = (w - 0.5) * 2 * theta_E

    all_loglike = Parallel(n_jobs=30)(delayed(compute_loglike)(row, T_0, Z_E, t_E_aff, bump_height, bump_width, dip_height, dip_width, del_go)\
                                       for _, row in df_led_off_valid.iterrows() )
    return np.sum(all_loglike)


## priors

In [10]:
T_0_bounds = [1e-3, 600e-3]
w_bounds = [0.3, 0.7]
t_E_aff_bounds = [0.001, 0.1]
bump_height_bounds = [0.02, 1]
bump_width_bounds = [0.1, 1]
dip_width_bounds = [0.01, 1]
dip_height_bounds = [0.001, 1]
del_go_bounds = [0.01, 0.15]

T_0_plausible_bounds = [80e-3, 200e-3]
w_plausible_bounds = [0.45, 0.55]
t_E_aff_plausible_bounds = [0.01, 0.05]
bump_height_plausible_bounds = [0.1, 0.5]
bump_width_plausible_bounds = [0.2, 0.3]
dip_width_plausible_bounds = [0.025, 0.05]
dip_height_plausible_bounds = [0.2, 0.5]
del_go_plausible_bounds = [0.05, 0.14]


In [11]:
def trapezoidal_logpdf(x, a, b, c, d):
    if x < a or x > d:
        return -np.inf  # Logarithm of zero
    area = ((b - a) + (d - c)) / 2 + (c - b)
    h_max = 1.0 / area  # Height of the trapezoid to normalize the area to 1
    
    if a <= x <= b:
        pdf_value = ((x - a) / (b - a)) * h_max
    elif b < x < c:
        pdf_value = h_max
    elif c <= x <= d:
        pdf_value = ((d - x) / (d - c)) * h_max
    else:
        pdf_value = 0.0  # This case is redundant due to the initial check

    if pdf_value <= 0.0:
        return -np.inf
    else:
        return np.log(pdf_value)

def vbmc_prior_fn(params):
    T_0, w, t_E_aff, bump_height, bump_width, dip_height, dip_width, del_go = params

    T_0_logpdf = trapezoidal_logpdf(
        T_0,
        T_0_bounds[0],
        T_0_plausible_bounds[0],
        T_0_plausible_bounds[1],
        T_0_bounds[1]
    )

    
    w_logpdf = trapezoidal_logpdf(
        w,
        w_bounds[0],
        w_plausible_bounds[0],
        w_plausible_bounds[1],
        w_bounds[1]
    )

    t_E_aff_logpdf = trapezoidal_logpdf(
        t_E_aff,
        t_E_aff_bounds[0],
        t_E_aff_plausible_bounds[0],
        t_E_aff_plausible_bounds[1],
        t_E_aff_bounds[1]
    )

    bump_height_logpdf = trapezoidal_logpdf(
        bump_height,
        bump_height_bounds[0],
        bump_height_plausible_bounds[0],
        bump_height_plausible_bounds[1],
        bump_height_bounds[1]
    )

    bump_width_logpdf = trapezoidal_logpdf(
        bump_width,
        bump_width_bounds[0],
        bump_width_plausible_bounds[0],
        bump_width_plausible_bounds[1],
        bump_width_bounds[1]
    )

    dip_height_logpdf = trapezoidal_logpdf(
        dip_height,
        dip_height_bounds[0],
        dip_height_plausible_bounds[0],
        dip_height_plausible_bounds[1],
        dip_height_bounds[1]
    )

    dip_width_logpdf = trapezoidal_logpdf(
        dip_width,
        dip_width_bounds[0],
        dip_width_plausible_bounds[0],
        dip_width_plausible_bounds[1],
        dip_width_bounds[1]
    )

    del_go_logpdf = trapezoidal_logpdf(
        del_go,
        del_go_bounds[0],
        del_go_plausible_bounds[0],
        del_go_plausible_bounds[1],
        del_go_bounds[1]
    )

    return (
        T_0_logpdf +
        w_logpdf +
        t_E_aff_logpdf +
        bump_height_logpdf +
        bump_width_logpdf +
        dip_height_logpdf +
        dip_width_logpdf + 
        del_go_logpdf
    )


## prior + loglike

In [12]:
def vbmc_joint_fn(params):
    priors = vbmc_prior_fn(params)
    loglike = vbmc_loglike_fn(params)

    return priors + loglike

# run vbmc

In [ ]:
# Add bounds for all parameters
lb = np.array([
    T_0_bounds[0],
    w_bounds[0],
    t_E_aff_bounds[0],
    bump_height_bounds[0],
    bump_width_bounds[0],
    dip_height_bounds[0],
    dip_width_bounds[0],
    del_go_bounds[0]
])

ub = np.array([
    T_0_bounds[1],
    w_bounds[1],
    t_E_aff_bounds[1],
    bump_height_bounds[1],
    bump_width_bounds[1],
    dip_height_bounds[1],
    dip_width_bounds[1],
    del_go_bounds[1]
])

plb = np.array([
    T_0_plausible_bounds[0],
    w_plausible_bounds[0],
    t_E_aff_plausible_bounds[0],
    bump_height_plausible_bounds[0],
    bump_width_plausible_bounds[0],
    dip_height_plausible_bounds[0],
    dip_width_plausible_bounds[0],
    del_go_plausible_bounds[0]
])

pub = np.array([
    T_0_plausible_bounds[1],
    w_plausible_bounds[1],
    t_E_aff_plausible_bounds[1],
    bump_height_plausible_bounds[1],
    bump_width_plausible_bounds[1],
    dip_height_plausible_bounds[1],
    dip_width_plausible_bounds[1],
    del_go_plausible_bounds[1]
])

# Initialize with random values within plausible bounds
np.random.seed(42)
# rate_lambda_0 = np.random.uniform(*rate_lambda_plausible_bounds) 
# rate_lambda_0 = 0.13 # hardcoded init
# T_0_0 = np.random.uniform(*T_0_plausible_bounds)
T_0_0 = 100e-3
# theta_E_0 = np.random.uniform(*theta_E_plausible_bounds)
# theta_E_0 = 45
# w_0 = np.random.uniform(*w_plausible_bounds)
w_0 = 0.5
# t_E_aff_0 = np.random.uniform(*t_E_aff_plausible_bounds)
t_E_aff_0 = 0.068
bump_height_0 = np.random.uniform(*bump_height_plausible_bounds)
bump_width_0 = np.random.uniform(*bump_width_plausible_bounds)
dip_height_0 = np.random.uniform(*dip_height_plausible_bounds)
dip_width_0 = np.random.uniform(*dip_width_plausible_bounds)
del_go_0 = 0.13

x_0 = np.array([
    T_0_0,
    w_0,
    t_E_aff_0,
    bump_height_0,
    bump_width_0,
    dip_height_0,
    dip_width_0,
    del_go_0
])

# Run VBMC
vbmc = VBMC(vbmc_joint_fn, x_0, lb, ub, plb, pub, options={'display': 'on'})
vp, results = vbmc.optimize()


Reshaping x0 to row vector.
Reshaping lower bounds to (1, 8).
Reshaping upper bounds to (1, 8).
Reshaping plausible lower bounds to (1, 8).
Reshaping plausible upper bounds to (1, 8).
vbmc:InitialPointsOutsidePB. The starting points X0 are not inside the provided plausible bounds PLB and PUB. Expanding the plausible bounds...
Beginning variational optimization assuming EXACT observations of the log-joint.
 Iteration  f-count    Mean[ELBO]    Std[ELBO]    sKL-iter[q]   K[q]  Convergence  Action
     0         10     -403539.73      1529.75    358170.88        2        inf     start warm-up
     1         15     -381794.05      1869.71     28443.95        2        inf     
     2         20     -367891.13      1627.12     19119.80        2   2.77e+05     
     3         25     -360170.84      1601.68      9575.32        2   1.44e+05     
     4         30     -359737.30       716.75       811.48        2   1.34e+04     
     5         35     -317232.73      4912.76    270035.70        2 

/home/rlab/raghavendra/ddm_data/.venv/lib/python3.12/site-packages/cma/evolution_strategy.py:3379: RuntimeWarning: invalid value encountered in scalar subtract
  current_fitness_range < opts['tolfunrel'] * (es.fit.median0 - es.fit.median_min),
/home/rlab/raghavendra/ddm_data/.venv/lib/python3.12/site-packages/cma/evolution_strategy.py:3379: RuntimeWarning: invalid value encountered in scalar multiply
  current_fitness_range < opts['tolfunrel'] * (es.fit.median0 - es.fit.median_min),


    11         65     -182961.02      6241.27    269275.50        2   3.37e+06     
    12         70     -170258.25      1849.69    253135.83        2   3.03e+06     
    13         75     -168170.95      1178.73       436.39        2    1.6e+04     
    14         80     -169341.40       942.28       137.23        2   8.66e+03     
    15         85     -168326.41      1366.20      2113.13        2   3.28e+04     
    16         90     -154073.67      9960.17    204732.65        2   2.49e+06     
    17         95     -154443.82      3555.69      9195.80        2   1.21e+05     
    18        100     -154418.83      2501.40      1751.55        2   2.91e+04     
    19        105     -152918.08      4903.65      5637.55        2   8.78e+04     
    20        110     -153658.60      2538.13      4397.25        2   6.28e+04     
    21        115     -150599.52      6530.69    990760.87        2   1.17e+07     
    22        120     -136212.83      2229.97 1315673109903.09        2   1.

In [ ]:
# vbmc.save('lamda_theta_fixed_NO_16_no_norm_del_go_vbmc_time_vary_all_phi.pkl', overwrite=True)

In [ ]:
# with open('lamda_theta_fixed_NO_16_no_norm_del_go_vbmc_time_vary_all_phi.pkl', 'rb') as f:
#     vp = pickle.load(f).vp

# continue vbmc run

In [ ]:
# load_filename = "NO_16_NO_NORM_TEMP_run.pkl"
# vbmc.save(load_filename, overwrite=True)
# new_options = {
#     'display': 'on', # Keep display on if you like
#     'max_fun_evals': 50
# }

# # Specify the filename where you saved the state


# # Load the VBMC instance from the file, applying the new options
# print(f"Loading VBMC state from {load_filename}...")
# vbmc = VBMC.load(load_filename, new_options=new_options)
# print("State loaded. Resuming optimization...")

# # Resume optimization - this will continue from where it left off
# vp, results = vbmc.optimize()


In [ ]:
## TEMP SAVE 
# vbmc.save('fit_all_phi_norm_and_time_vary.pkl')

#### ********* READING VP from OLD PICKLE FILE ******************

In [ ]:
# with open('fit_all_phi_norm_and_time_vary.pkl', 'rb') as f:
#     vp = pickle.load(f).vp

# corner

In [ ]:
# Sample from the VBMC posterior (returns tuple: samples, log weights)
vp_samples = vp.sample(int(1e5))[0]

# Convert T_0 to ms
vp_samples[:, 0] *= 1e3  # T_0 is at index 1

# Parameter labels (order matters!)
# param_labels = [r'$T_0$ (ms)', r'$\theta_E$', r'$w$', r'$t_{E}^{aff}$', 'bump_height', 'bump_width', 'dip_height', 'dip_width', 'del_go']
param_labels = ['T0', 'w', 't_E', 'bump_height', 'bump_width', 'dip_height', 'dip_width', 'del_go']

# Compute 1st and 99th percentiles for each param to restrict range
percentiles = np.percentile(vp_samples, [1, 99], axis=0)
_ranges = [(percentiles[0, i], percentiles[1, i]) for i in range(vp_samples.shape[1])]

# Create the corner plot
fig = corner.corner(
    vp_samples,
    labels=param_labels,
    show_titles=True,
    quantiles=[0.025, 0.5, 0.975],
    range=_ranges,
    title_fmt=".3f"
)

vp_samples[:, 0] /= 1e3  # T_0 is at index 1


plt.show()


# diagnostics

## params

In [ ]:
T_0 = vp_samples[:, 0].mean()

w = vp_samples[:, 1].mean()
Z_E = (w - 0.5) * 2 * theta_E

t_E_aff = vp_samples[:, 2].mean()
bump_height = vp_samples[:, 3].mean()
bump_width = vp_samples[:, 4].mean()
dip_height = vp_samples[:, 5].mean()
dip_width = vp_samples[:, 6].mean()
del_go = vp_samples[:, 7].mean()




# Print them out
print("Posterior Means:")
print(f"rate_lambda  = {rate_lambda:.5f}")
print(f"T_0 (ms)      = {1e3*T_0:.5f}")
print(f"theta_E       = {theta_E:.5f}")
print(f"Z_E           = {Z_E:.5f}")
print(f"t_E_aff       = {1e3*t_E_aff:.5f} ms")
print(f"rate_norm_l   = {rate_norm_l:.5f}")
print(f'bump height = {bump_height :.5f}')
print(f'bump_width = {bump_width :.5f}')
print(f'dip_height = {dip_height :.5f}')
print(f'dip_width = {dip_width :.5f}')
print(f'del go = {del_go :.5f}')

In [ ]:
phi_params = {
    'h1': bump_width,
    'a1': bump_height,
    'h2': dip_width,
    'a2': dip_height,
    'b1': bump_offset
}
phi_params_obj = SimpleNamespace(**phi_params)

## simualte

In [ ]:
# sample t-stim
N_sim = int(1e6)

t_stim_samples = df['intended_fix'].sample(N_sim, replace=True).values
ABL_samples = df['ABL'].sample(N_sim, replace=True).values
ILD_samples = df['ILD'].sample(N_sim, replace=True).values

N_print = int(N_sim / 5)
dt  = 1e-4

rate_norm_l = 0 # NO NORM

sim_results = Parallel(n_jobs=30)(
    delayed(psiam_tied_data_gen_wrapper_rate_norm_time_vary_fn)(
        V_A, theta_A, ABL_samples[iter_num], ILD_samples[iter_num], rate_lambda, T_0, theta_E, Z_E, t_A_aff, t_E_aff, del_go, 
        t_stim_samples[iter_num], rate_norm_l, iter_num, N_print, phi_params_obj, dt
    ) for iter_num in tqdm(range(N_sim))
)


# temp - *** ADD 16 back *****

In [ ]:
# LED off rows
df_led_off = df[ df['LED_trial'] == 0 ]
print(f'len df_led_off = {len(df_led_off)}')

# > 0 and < 1s valid rt 
df_led_off_valid = df_led_off[
    (df_led_off['timed_fix'] - df_led_off['intended_fix'] > 0) &
    (df_led_off['timed_fix'] - df_led_off['intended_fix'] < 1)
]

df_led_off_valid = df_led_off_valid[df_led_off_valid['response_poke'].isin([2,3])]

## prepare valid sim df, data df

In [ ]:
sim_results_df = pd.DataFrame(sim_results)
sim_results_df_valid = sim_results_df[
    (sim_results_df['rt'] > sim_results_df['t_stim']) &
    (sim_results_df['rt'] - sim_results_df['t_stim'] < 1)
].copy()
sim_results_df_valid.loc[:, 'correct'] = (sim_results_df_valid['ILD'] * sim_results_df_valid['choice']).apply(lambda x: 1 if x > 0 else 0)


df_led_off_valid_renamed = df_led_off_valid.rename(columns = {
    'timed_fix': 'rt',
    'intended_fix': 't_stim'
}).copy()

sim_df_1 = sim_results_df_valid.copy()
data_df_1 = df_led_off_valid_renamed.copy()

## plots

In [ ]:
bw = 0.02
bins = np.arange(0, 1, bw)
bin_centers = bins[:-1] + (0.5 * bw)

n_rows = len(ILD_arr)
n_cols = len(ABL_arr)

fig, axs = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 3 * n_rows), sharey='row')

for i_idx, ILD in enumerate(ILD_arr):
    for a_idx, ABL in enumerate(ABL_arr):
        ax = axs[i_idx, a_idx] if n_rows > 1 else axs[a_idx]

        sim_df_1_ABL_ILD = sim_df_1[(sim_df_1['ABL'] == ABL) & (sim_df_1['ILD'] == ILD)]
        data_df_1_ABL_ILD = data_df_1[(data_df_1['ABL'] == ABL) & (data_df_1['ILD'] == ILD)]

        sim_up = sim_df_1_ABL_ILD[sim_df_1_ABL_ILD['choice'] == 1]
        sim_down = sim_df_1_ABL_ILD[sim_df_1_ABL_ILD['choice'] == -1]
        data_up = data_df_1_ABL_ILD[data_df_1_ABL_ILD['choice'] == 1]
        data_down = data_df_1_ABL_ILD[data_df_1_ABL_ILD['choice'] == -1]

        sim_up_rt = sim_up['rt'] - sim_up['t_stim']
        sim_down_rt = sim_down['rt'] - sim_down['t_stim']
        data_up_rt = data_up['rt'] - data_up['t_stim']
        data_down_rt = data_down['rt'] - data_down['t_stim']

        sim_up_hist, _ = np.histogram(sim_up_rt, bins=bins, density=True)
        sim_down_hist, _ = np.histogram(sim_down_rt, bins=bins, density=True)
        data_up_hist, _ = np.histogram(data_up_rt, bins=bins, density=True)
        data_down_hist, _ = np.histogram(data_down_rt, bins=bins, density=True)

        # Normalize histograms by proportion of trials
        sim_up_hist *= len(sim_up) / len(sim_df_1_ABL_ILD)
        sim_down_hist *= len(sim_down) / len(sim_df_1_ABL_ILD)
        data_up_hist *= len(data_up) / len(data_df_1_ABL_ILD)
        data_down_hist *= len(data_down) / len(data_df_1_ABL_ILD)

        # Plot
        ax.plot(bin_centers, data_up_hist, color='b', label='Data' if (i_idx == 0 and a_idx == 0) else "")
        ax.plot(bin_centers, -data_down_hist, color='b')
        ax.plot(bin_centers, sim_up_hist, color='r', label='Sim' if (i_idx == 0 and a_idx == 0) else "")
        ax.plot(bin_centers, -sim_down_hist, color='r')

        # Compute fractions
        data_total = len(data_df_1_ABL_ILD)
        sim_total = len(sim_df_1_ABL_ILD)
        data_up_frac = len(data_up) / data_total if data_total else 0
        data_down_frac = len(data_down) / data_total if data_total else 0
        sim_up_frac = len(sim_up) / sim_total if sim_total else 0
        sim_down_frac = len(sim_down) / sim_total if sim_total else 0

        ax.set_title(
            f"ABL: {ABL}, ILD: {ILD}\n"
            f"Data,Sim: (+{data_up_frac:.2f},+{sim_up_frac:.2f}), "
            f"(-{data_down_frac:.2f},-{sim_down_frac:.2f})"
        )
        
        ax.axhline(0, color='k', linewidth=0.5)
        ax.set_xlim([0, 0.7])
        if a_idx == 0:
            ax.set_ylabel("Density (Up / Down flipped)")
        if i_idx == n_rows - 1:
            ax.set_xlabel("RT (s)")

fig.tight_layout()
fig.legend(loc='upper right')
plt.show()


## tacho

In [ ]:
bw = 0.02
bins = np.arange(0, 1, bw)
bin_centers = bins[:-1] + (0.5 * bw)

def plot_tacho(df_1):
    df_1 = df_1.copy()
    df_1['RT_bin'] = pd.cut(df_1['rt'] - df_1['t_stim'], bins=bins, include_lowest=True)
    grouped = df_1.groupby('RT_bin', observed=False)['correct'].agg(['mean', 'count'])
    grouped['bin_mid'] = grouped.index.map(lambda x: x.mid)
    return grouped['bin_mid'], grouped['mean']

n_rows = len(ILD_arr)
n_cols = len(ABL_arr)

# === Define fig2 ===
fig2, axs = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 3 * n_rows), sharey='row')

for i_idx, ILD in enumerate(ILD_arr):
    for a_idx, ABL in enumerate(ABL_arr):
        ax = axs[i_idx, a_idx] if n_rows > 1 else axs[a_idx]

        sim_df_1_ABL_ILD = sim_df_1[(sim_df_1['ABL'] == ABL) & (sim_df_1['ILD'] == ILD)]
        data_df_1_ABL_ILD = data_df_1[(data_df_1['ABL'] == ABL) & (data_df_1['ILD'] == ILD)]

        sim_tacho_x, sim_tacho_y = plot_tacho(sim_df_1_ABL_ILD)
        data_tacho_x, data_tacho_y = plot_tacho(data_df_1_ABL_ILD)

        # Plotting
        ax.plot(data_tacho_x, data_tacho_y, color='b', label='Data' if (i_idx == 0 and a_idx == 0) else "")
        ax.plot(sim_tacho_x, sim_tacho_y, color='r', label='Sim' if (i_idx == 0 and a_idx == 0) else "")

        ax.set_ylim([0.5, 1.05])
        ax.set_xlim([0, 0.7])
        ax.set_title(f"ABL: {ABL}, ILD: {ILD}")
        if a_idx == 0:
            ax.set_ylabel("P(correct)")
        if i_idx == n_rows - 1:
            ax.set_xlabel("RT (s)")

fig2.tight_layout()
fig2.legend(loc='upper right')
plt.show()


## grand rtd, psycho, tacho

In [ ]:
def grand_rtd(df_1):
    df_1_rt = df_1['rt'] - df_1['t_stim']
    rt_hist, _ = np.histogram(df_1_rt, bins=bins, density=True)
    return rt_hist

def plot_psycho(df_1):
    prob_choice_dict = {}

    all_ABL = np.sort(df_1['ABL'].unique())
    all_ILD = np.sort(df_1['ILD'].unique())

    for abl in all_ABL:
        filtered_df = df_1[df_1['ABL'] == abl]
        prob_choice_dict[abl] = [
            sum(filtered_df[filtered_df['ILD'] == ild]['choice'] == 1) / len(filtered_df[filtered_df['ILD'] == ild])
            for ild in all_ILD
        ]

    return prob_choice_dict

# === Define fig3 ===
fig3, axes = plt.subplots(1, 3, figsize=(15, 4))

# === Grand RTD ===
axes[0].plot(bin_centers, grand_rtd(data_df_1), color='b', label='data')
axes[0].plot(bin_centers, grand_rtd(sim_df_1), color='r', label='sim')
axes[0].legend()
axes[0].set_xlabel('rt wrt stim')
axes[0].set_ylabel('density')
axes[0].set_title('Grand RTD')

# === Grand Psychometric ===
data_psycho = plot_psycho(data_df_1)
sim_psycho = plot_psycho(sim_df_1)

colors = ['r', 'b', 'g']  # Adjust colors for your ABLs
for i, ABL in enumerate(ABL_arr):
    axes[1].plot(ILD_arr, data_psycho[ABL], color=colors[i], label=f'data ABL={ABL}', marker='o', linestyle='None')
    axes[1].plot(ILD_arr, sim_psycho[ABL], color=colors[i], linestyle='-')

axes[1].legend()
axes[1].set_xlabel('ILD')
axes[1].set_ylabel('P(right)')
axes[1].set_title('Grand Psychometric')

# === Grand Tacho ===
data_tacho_x, data_tacho_y = plot_tacho(data_df_1)
sim_tacho_x, sim_tacho_y = plot_tacho(sim_df_1)

axes[2].plot(data_tacho_x, data_tacho_y, color='b', label='data')
axes[2].plot(sim_tacho_x, sim_tacho_y, color='r', label='sim')
# axes[2].axvline(del_go - t_E_aff, color='k', alpha=0.3)
axes[2].legend()
axes[2].set_xlabel('rt wrt stim')
axes[2].set_ylabel('acc')
axes[2].set_title('Grand Tacho')
axes[2].set_ylim(0.5, 1)

fig3.tight_layout()
plt.show()


## save diagnostics in pdf, docx

In [ ]:
import os
from matplotlib.backends.backend_pdf import PdfPages
from docx import Document
from docx.shared import Inches

# Set your filename prefix
output_filename = 'DEL_GO_150ms_NO_16_NO_NORM_only_time_vary'

# Ensure output directory
os.makedirs('outputs', exist_ok=True)

# === Save individual figures as PNGs ===
fig1_path = f'outputs/{output_filename}_updown_hist.png'
fig2_path = f'outputs/{output_filename}_tacho.png'
fig3_path = f'outputs/{output_filename}_grand_summary.png'

fig.savefig(fig1_path)
fig2.savefig(fig2_path)
fig3.savefig(fig3_path)

# === Create PDF with all three figures ===
pdf_path = f'outputs/{output_filename}.pdf'
with PdfPages(pdf_path) as pdf:
    for fig_item in [fig, fig2, fig3]:
        pdf.savefig(fig_item)

# === Create DOCX with all three figures ===
doc = Document()
doc.add_heading('RTD and Tacho Analysis Results', 0)

for img_path in [fig1_path, fig2_path, fig3_path]:
    doc.add_page_break()
    doc.add_picture(img_path, width=Inches(6.5))

docx_path = f'outputs/{output_filename}.docx'
doc.save(docx_path)

print(f"✅ Saved PDF to: {pdf_path}")
print(f"✅ Saved DOCX to: {docx_path}")


## mu * phi(t), sigma^2 * phi(t) for each stim

In [ ]:
def mu_phi(t_pts, ABL, ILD):
    chi = 17.37

    lambda_ABL_term = (10 ** (rate_lambda * (1 - rate_norm_l) * ABL / 20))
    lambda_ILD_arg = rate_lambda * ILD / chi
    lambda_ILD_L_arg = rate_lambda * rate_norm_l * ILD / chi

    # Vectorized phi_t over t_pts
    phi_t = phi_t_fn(t_pts, phi_params_obj.h1, phi_params_obj.a1, phi_params_obj.b1, phi_params_obj.h2, phi_params_obj.a2)
    
    mu = (1 / T_0) * lambda_ABL_term * (np.sinh(lambda_ILD_arg) / np.cosh(lambda_ILD_L_arg)) * phi_t
    
    return mu * phi_t  # element-wise product for each t in t_pts


def sigma_sq_phi(t_pts, ABL, ILD):
    chi = 17.37

    lambda_ABL_term = (10 ** (rate_lambda * (1 - rate_norm_l) * ABL / 20))
    lambda_ILD_arg = rate_lambda * ILD / chi
    lambda_ILD_L_arg = rate_lambda * rate_norm_l * ILD / chi

    # Vectorized phi_t over t_pts
    phi_t = phi_t_fn(t_pts, phi_params_obj.h1, phi_params_obj.a1, phi_params_obj.b1, phi_params_obj.h2, phi_params_obj.a2)
    
    sigma_sq = (1/T_0) * lambda_ABL_term * ( np.cosh(lambda_ILD_arg) / np.cosh(lambda_ILD_L_arg) ) * phi_t
    
    return sigma_sq * phi_t  # element-wise product for each t in t_pts


## mu * phi(t)

In [ ]:
t_pts = np.arange(0, 1, 0.02)
ILD_arr_pos = ILD_arr[ILD_arr > 0]

colors = ['r', 'b', 'g']  # Assuming len(ABL_arr) == 3

# First, precompute y-limits across all ILDs and ABLs
all_mu_vals = []
for ILD in ILD_arr_pos:
    for ABL in ABL_arr:
        all_mu_vals.append(mu_phi(t_pts, ABL, ILD))

y_min = min(np.min(mu) for mu in all_mu_vals)
y_max = max(np.max(mu) for mu in all_mu_vals)

# Now, create subplots
plt.figure(figsize=(4 * len(ILD_arr_pos), 4))
for i_idx, ILD in enumerate(ILD_arr_pos):
    ax = plt.subplot(1, len(ILD_arr_pos), i_idx + 1)
    
    for a_idx, ABL in enumerate(ABL_arr):
        ax.plot(t_pts, mu_phi(t_pts, ABL, ILD), label=f'ABL={ABL}', color=colors[a_idx])
    
    ax.set_title(f'ILD = {ILD}')
    ax.set_ylim([y_min, y_max])
    
    if i_idx == 0:
        ax.legend()
    else:
        ax.legend().set_visible(False)

    ax.set_xlabel('Time (s)')
    if i_idx == 0:
        ax.set_ylabel('μ·φ(t)')

plt.tight_layout()
plt.show()


## sigma^2 x phi(t)

In [ ]:
t_pts = np.arange(0, 1, 0.02)
ILD_arr_pos = ILD_arr[ILD_arr > 0]

colors = ['r', 'b', 'g']  # Assuming len(ABL_arr) == 3

# Compute y-limits for consistent scale
all_sigma_vals = []
for ILD in ILD_arr_pos:
    for ABL in ABL_arr:
        all_sigma_vals.append(sigma_sq_phi(t_pts, ABL, ILD))

y_min = min(np.min(sig) for sig in all_sigma_vals)
y_max = max(np.max(sig) for sig in all_sigma_vals)

# Create subplots
plt.figure(figsize=(4 * len(ILD_arr_pos), 4))
for i_idx, ILD in enumerate(ILD_arr_pos):
    ax = plt.subplot(1, len(ILD_arr_pos), i_idx + 1)
    
    for a_idx, ABL in enumerate(ABL_arr):
        ax.plot(t_pts, sigma_sq_phi(t_pts, ABL, ILD), label=f'ABL={ABL}', color=colors[a_idx])
    
    ax.set_title(f'ILD = {ILD}')
    ax.set_ylim([y_min, y_max])
    
    if i_idx == 0:
        ax.legend()
    else:
        ax.legend().set_visible(False)

    ax.set_xlabel('Time (s)')
    if i_idx == 0:
        ax.set_ylabel('σ²·φ(t)')

plt.tight_layout()
plt.show()


# up and down RTD

In [ ]:
N_theory = int(1e3)
t_stim_and_led_tuple = [(row['intended_fix'], row['intended_fix'] - row['LED_onset_time']) for _, row in df.iterrows()]
random_indices = np.random.randint(0, len(t_stim_and_led_tuple), N_theory)
t_pts = np.arange(-1, 2, 0.001)

P_A_samples = np.zeros((N_theory, len(t_pts)))
for idx in range(N_theory):
    t_stim, t_LED = t_stim_and_led_tuple[random_indices[idx]]
    pdf = rho_A_t_VEC_fn(t_pts - t_A_aff + t_stim, V_A, theta_A)
    P_A_samples[idx, :] = pdf

P_A_samples_mean = np.mean(P_A_samples, axis=0)
C_A_mean = cumtrapz(P_A_samples_mean, t_pts, initial=0)

# per stim, up and down RTD

In [ ]:
# Create figure and axes row by row to enable row-wise shared Y axes
fig = plt.figure(figsize=(18, 24))
axes = []

for i in range(10):  # 10 rows
    row_axes = []
    for j in range(3):  # 3 columns
        ax = fig.add_subplot(10, 3, i * 3 + j + 1, sharey=row_axes[0] if row_axes else None)
        row_axes.append(ax)
    axes.append(row_axes)

fig.subplots_adjust(hspace=0.8, wspace=0.4)

bin_size = 0.02
bins = np.arange(-1, 2, bin_size)
bin_centers = bins[:-1] + (bin_size / 2)
t_pts = np.arange(-1, 2, 0.001)

phi_params = {
    'h1': bump_width,
    'a1': bump_height,
    'h2': dip_width,
    'a2': dip_height,
    'b1': bump_offset
}
phi_params_obj = SimpleNamespace(**phi_params)

# Plotting
for a_idx, ABL in enumerate(ABL_arr):
    for i_idx, ILD in enumerate(ILD_arr):
        ax = axes[i_idx][a_idx]  # axes[row][col] = (ILD, ABL)

        # data
        # ONLY valid
        # df_ABL_ILD = df_led_off_valid[
        #     (df_led_off_valid['ABL'] == ABL) & (df_led_off_valid['ILD'] == ILD)]
        # valid + aborts
        df_led_off_abort_and_valid = df_led_off[(df_led_off['abort_event'] == 3) | (df_led_off['response_poke'].isin([2,3]))]
        mask_invalid = ~df_led_off_abort_and_valid['response_poke'].isin([2, 3])
        # Step 2: Assign random values (2 or 3 with 50% chance) to those rows
        df_led_off_abort_and_valid.loc[mask_invalid, 'response_poke'] = np.random.choice([2, 3], size=mask_invalid.sum())

        df_ABL_ILD = df_led_off_abort_and_valid[
            (df_led_off_abort_and_valid['ABL'] == ABL) & (df_led_off_abort_and_valid['ILD'] == ILD)]
        df_ABL_ILD_up = df_ABL_ILD[df_ABL_ILD['response_poke'] == 3]
        df_ABL_ILD_down = df_ABL_ILD[df_ABL_ILD['response_poke'] == 2]

        df_ABL_ILD_up_rt = df_ABL_ILD_up['timed_fix'] - df_ABL_ILD_up['intended_fix']
        df_ABL_ILD_down_rt = df_ABL_ILD_down['timed_fix'] - df_ABL_ILD_down['intended_fix']

        data_up_rt_hist, _ = np.histogram(df_ABL_ILD_up_rt, bins=bins, density=True)
        data_down_rt_hist, _ = np.histogram(df_ABL_ILD_down_rt, bins=bins, density=True)

        data_frac_up = len(df_ABL_ILD_up) / len(df_ABL_ILD)
        data_frac_down = len(df_ABL_ILD_down) / len(df_ABL_ILD)

        # theory
        theory_ABL_ILD_up = np.zeros_like(t_pts)
        theory_ABL_ILD_down = np.zeros_like(t_pts)

        for idx, t in enumerate(t_pts):
            P_A = P_A_samples_mean[idx]
            C_A = C_A_mean[idx]
            theory_ABL_ILD_up[idx] = up_or_down_RTs_fit_wrt_stim_fn(
                t, 1, P_A, C_A,
                t_stim, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_E_aff, del_go,
                phi_params_obj, rate_norm_l, is_norm, is_time_vary, K_max)

            theory_ABL_ILD_down[idx] = up_or_down_RTs_fit_wrt_stim_fn(
                t, -1, P_A, C_A,
                t_stim, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_E_aff, del_go,
                phi_params_obj, rate_norm_l, is_norm, is_time_vary, K_max)

        ax.plot(bin_centers, data_up_rt_hist * data_frac_up, 'b--', label='Data Up' if i_idx == 0 and a_idx == 0 else "")
        ax.plot(bin_centers, -data_down_rt_hist * data_frac_down, 'b--', label='Data Down' if i_idx == 0 and a_idx == 0 else "")
        ax.plot(t_pts, theory_ABL_ILD_up, 'r-', label='Theory Up' if i_idx == 0 and a_idx == 0 else "")
        ax.plot(t_pts, -theory_ABL_ILD_down, 'r-', label='Theory Down' if i_idx == 0 and a_idx == 0 else "")
        print(f'### ABL = {ABL}, ILD = {ILD} ###')
        print(f'theory areas:')
        theory_area_up = trapz(theory_ABL_ILD_up, t_pts); theory_area_down = trapz(theory_ABL_ILD_down, t_pts)
        print(f'Up: {theory_area_up :.3f}, down={theory_area_down :.3f}, up + down = {(theory_area_up  + theory_area_down) :.3f}')
        print(f'data areas:')
        print(f'up = {data_frac_up :.3f}, down = {data_frac_down :.3f}, up + down = {(data_frac_up + data_frac_down) :.3f}')
        ax.set_title(f'ABL={ABL}, ILD={ILD}', fontsize=9)
        ax.axhline(0, color='black', linewidth=0.5)
        ax.set_xlim(-0.2, 0.7)

# Add a single legend outside the plot
handles, labels = axes[0][0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right', fontsize=10)

plt.tight_layout(rect=[0, 0, 0.98, 1])
plt.show()


# tacho

In [ ]:
import matplotlib.pyplot as plt

# Prepare the figure and axes
fig, axes = plt.subplots(nrows=10, ncols=3, figsize=(18, 24), sharex=True, sharey=True)
fig.subplots_adjust(hspace=0.5, wspace=0.3)
bin_size = 0.02
bins = np.arange(-1, 2, bin_size)
bin_centers = bins[:-1] + (bin_size / 2)
t_pts = np.arange(-1, 2, 0.001)


for a_idx, ABL in enumerate(ABL_arr):
    for i_idx, ILD in enumerate(ILD_arr):
        ax = axes[i_idx, a_idx]  # (row=ILD, col=ABL)

        # data
        df_led_off_abort_and_valid = df_led_off[(df_led_off['abort_event'] == 3) | (df_led_off['response_poke'].isin([2,3]))]
        mask_invalid = ~df_led_off_abort_and_valid['response_poke'].isin([2, 3])
        # Step 2: Assign random values (2 or 3 with 50% chance) to those rows
        df_led_off_abort_and_valid.loc[mask_invalid, 'response_poke'] = np.random.choice([2, 3], size=mask_invalid.sum())

        df_ABL_ILD = df_led_off_abort_and_valid[
            (df_led_off_abort_and_valid['ABL'] == ABL) & (df_led_off_abort_and_valid['ILD'] == ILD)].copy()
        
        # df_ABL_ILD = df_led_off_valid[
        #     (df_led_off_valid['ABL'] == ABL) & (df_led_off_valid['ILD'] == ILD)].copy()
        
        df_ABL_ILD.loc[:, 'RT'] = df_ABL_ILD['timed_fix'] - df_ABL_ILD['intended_fix']
        df_ABL_ILD.loc[:, 'is_correct'] = (
                    df_ABL_ILD['ILD'] * (2 * df_ABL_ILD['response_poke'] - 5)
                ) > 0
        df_ABL_ILD.loc[:, 'rt_bin'] = pd.cut(
                df_ABL_ILD['RT'], bins=bins, include_lowest=True
            )
        tachometric_curve = df_ABL_ILD.groupby('rt_bin', observed=False)['is_correct'].mean()


        # theory
        theory_ABL_ILD_up = np.zeros_like(t_pts)
        theory_ABL_ILD_down = np.zeros_like(t_pts)
        theory_tacho = np.zeros_like(t_pts)
        for idx, t in enumerate(t_pts):
            P_A = P_A_samples_mean[idx]
            C_A = C_A_mean[idx]
            theory_ABL_ILD_up[idx] = up_or_down_RTs_fit_wrt_stim_fn(
                t, 1,
                P_A, C_A,
                t_stim, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_E_aff, del_go,
                phi_params_obj, rate_norm_l,
                is_norm, is_time_vary, K_max)

            theory_ABL_ILD_down[idx] = up_or_down_RTs_fit_wrt_stim_fn(
                t, -1,
                P_A, C_A,
                t_stim, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_E_aff, del_go,
                phi_params_obj, rate_norm_l,
                is_norm, is_time_vary, K_max)

            if ILD > 0:
                theory_tacho[idx] = theory_ABL_ILD_up[idx] / (theory_ABL_ILD_up[idx] + theory_ABL_ILD_down[idx] + 1e-10)
            else:
                theory_tacho[idx] = theory_ABL_ILD_down[idx] / (theory_ABL_ILD_up[idx] + theory_ABL_ILD_down[idx] + 1e-10)
        

        ax.plot(bin_centers, tachometric_curve, 'b--')
        ax.plot(t_pts, theory_tacho, 'r-')
        ax.set_title(f'ABL={ABL}, ILD={ILD}', fontsize=9)
        ax.axhline(0, color='black', linewidth=0.5)
        ax.set_xlim(0, 0.7)
        ax.set_ylim(0.5, 1.05)

# Add shared axis labels

# Add a single legend outside the plot
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right', fontsize=10)

plt.tight_layout(rect=[0, 0, 0.98, 1])  # leave space for legend
plt.show()


# check with simulations 

In [ ]:
# sample t-stim
N_sim = int(1e6)

t_stim_samples = df['intended_fix'].sample(N_sim, replace=True).values
ABL_samples = df['ABL'].sample(N_sim, replace=True).values
ILD_samples = df['ILD'].sample(N_sim, replace=True).values

N_print = int(N_sim / 5)
dt  = 1e-4

sim_results = Parallel(n_jobs=30)(
    delayed(psiam_tied_data_gen_wrapper_rate_norm_time_vary_fn)(
        V_A, theta_A, ABL_samples[iter_num], ILD_samples[iter_num], rate_lambda, T_0, theta_E, Z_E, t_A_aff, t_E_aff, del_go, 
        t_stim_samples[iter_num], rate_norm_l, iter_num, N_print, phi_params_obj, dt
    ) for iter_num in tqdm(range(N_sim))
)

# Grand RT, Grand Tacho, Psycho

In [ ]:
sim_results_df = pd.DataFrame(sim_results)
sim_results_df_valid = sim_results_df[sim_results_df['rt'] > sim_results_df['t_stim']]
sim_results_df_valid_less_than_1 = sim_results_df_valid[sim_results_df_valid['rt'] - sim_results_df_valid['t_stim'] < 1].copy()

df_led_off_valid.loc[:,'choice'] = 2*df_led_off_valid['response_poke'] - 5
df_led_off_valid_renamed = df_led_off_valid.rename(columns={
    'timed_fix': 'rt',
    'intended_fix': 't_stim'
}).copy()

df_led_off_valid_renamed.loc[:,'correct'] = (df_led_off_valid_renamed['choice'] * df_led_off_valid_renamed['ILD'] > 0).astype(int)
sim_results_df_valid_less_than_1.loc[:,'correct'] = (sim_results_df_valid_less_than_1['choice'] * sim_results_df_valid_less_than_1['ILD'] > 0).astype(int)

In [ ]:
bw = 0.02
bins = np.arange(0, 1, bw)
bin_centers = bins[:-1] + 0.5*bw
def grand_rtd(df_1):
    df_1_rt = df_1['rt'] - df_1['t_stim']
    rt_hist, _ = np.histogram(df_1_rt, bins=bins, density=True)
    return rt_hist

def plot_psycho(df_1):
    prob_choice_dict = {}

    all_ABL = np.sort(df_1['ABL'].unique())
    all_ILD = np.sort(df_1['ILD'].unique())

    for abl in all_ABL:
        filtered_df = df_1[df_1['ABL'] == abl]
        prob_choice_dict[abl] = [sum(filtered_df[filtered_df['ILD'] == ild]['choice'] == 1) / len(filtered_df[filtered_df['ILD'] == ild]) for ild in all_ILD]

    return prob_choice_dict

def plot_tacho(df_1):
    # prob of correct vs RT
    df_1.loc[:, 'RT_bin'] = pd.cut(df_1['rt'] - df_1['t_stim'], bins=bins, include_lowest=True)
    grouped_by_rt_bin = df_1.groupby('RT_bin', observed=False)['correct'].agg(['mean', 'count'])
    grouped_by_rt_bin['bin_mid'] = grouped_by_rt_bin.index.map(lambda x: x.mid)
    return grouped_by_rt_bin['bin_mid'], grouped_by_rt_bin['mean']

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with 3 subplots in a single row
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# === grand RTD ===
axes[0].plot(bin_centers, grand_rtd(df_led_off_valid_renamed), color='b', label='data')
axes[0].plot(bin_centers, grand_rtd(sim_results_df_valid_less_than_1), color='r', label='sim')
axes[0].legend()
axes[0].set_xlabel('rt wrt stim')
axes[0].set_ylabel('density')
axes[0].set_title('Grand RTD')

# === grand psycho ===
data_psycho = plot_psycho(df_led_off_valid_renamed)
sim_psycho = plot_psycho(sim_results_df_valid_less_than_1)

colors = ['r', 'b', 'g']  # Define colors for each ABL
for i, ABL in enumerate(ABL_arr):
    axes[1].plot(ILD_arr, data_psycho[ABL], color=colors[i], label=f'data ABL={ABL}', marker='o', linestyle='None')
    axes[1].plot(ILD_arr, sim_psycho[ABL], color=colors[i], label=f'sim ABL={ABL}', linestyle='-')

axes[1].legend()
axes[1].set_xlabel('ILD')
axes[1].set_ylabel('P(right)')
axes[1].set_title('Grand Psychometric')

# === grand tacho ===
data_tacho_x, data_tacho_y = plot_tacho(df_led_off_valid_renamed)
sim_tacho_x, sim_tacho_y = plot_tacho(sim_results_df_valid_less_than_1)

axes[2].plot(data_tacho_x, data_tacho_y, color='b', label='data')
axes[2].plot(sim_tacho_x, sim_tacho_y, color='r', label='sim')
axes[2].legend()
axes[2].set_xlabel('rt wrt stim')
axes[2].set_ylabel('acc')
axes[2].set_title('Grand Tacho')
axes[2].set_ylim(0.5, 1);


In [ ]:
ABL = 20; ILD = 1
sim_results_df_ABL_ILD = sim_results_df[(sim_results_df['ABL'] == ABL) & (sim_results_df['ILD'] == ILD)]
bins = np.arange(-1, 2, 0.02)
rt_ABL_ILD = sim_results_df_ABL_ILD['rt'] - sim_results_df_ABL_ILD['t_stim']


theory_total = np.zeros_like(t_pts)
theory_up = np.zeros_like(t_pts)
theory_down = np.zeros_like(t_pts)

for idx, t in enumerate(t_pts):
    P_A = P_A_samples_mean[idx]
    C_A = C_A_mean[idx]
    up = up_or_down_RTs_fit_wrt_stim_fn(
        t, 1, P_A, C_A,
        np.nan, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_E_aff, del_go,
        phi_params_obj, rate_norm_l, is_norm, is_time_vary, K_max)

    down = up_or_down_RTs_fit_wrt_stim_fn(
        t, -1, P_A, C_A,
        np.nan, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_E_aff, del_go,
        phi_params_obj, rate_norm_l, is_norm, is_time_vary, K_max)

    theory_up[idx]  = up
    theory_down[idx] = down  
    theory_total[idx] = up + down


plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(t_pts, theory_total, ls='--')
plt.hist(rt_ABL_ILD, bins=bins, density=True, histtype='step');

plt.subplot(1,2,2)
sim_results_df_ABL_ILD_up = sim_results_df_ABL_ILD[sim_results_df_ABL_ILD['choice'] == 1]
sim_results_df_ABL_ILD_down = sim_results_df_ABL_ILD[sim_results_df_ABL_ILD['choice'] == -1]

up_rt_hist, _ = np.histogram(sim_results_df_ABL_ILD_up['rt'] - sim_results_df_ABL_ILD_up['t_stim'], bins=bins, density=True)
down_rt_hist, _ = np.histogram(sim_results_df_ABL_ILD_down['rt'] - sim_results_df_ABL_ILD_down['t_stim'], bins=bins, density=True)


bin_centers = bins[:-1] + 0.5 * 0.02
plt.plot(bin_centers, up_rt_hist * len(sim_results_df_ABL_ILD_up)/len(sim_results_df_ABL_ILD))
plt.plot(bin_centers, -1 *  down_rt_hist * len(sim_results_df_ABL_ILD_down)/len(sim_results_df_ABL_ILD))


plt.plot(t_pts, theory_up, ls='--')
plt.plot(t_pts, -theory_down, ls='--')


In [ ]:
# Convert t_pts to numpy array if it's not already
t_pts = np.array(t_pts)

# Define masks for the integration ranges
mask_full = (t_pts >= -1) & (t_pts <= 2)
mask_pos = (t_pts >= 0) & (t_pts <= 2)

# Compute areas under the curve for full range [-1, 2]
area_up_full = np.trapz(theory_up[mask_full], t_pts[mask_full])
area_down_full = np.trapz(theory_down[mask_full], t_pts[mask_full])
area_total_full = area_up_full + area_down_full

# Compute areas under the curve for positive range [0, 2]
area_up_pos = np.trapz(theory_up[mask_pos], t_pts[mask_pos])
area_down_pos = np.trapz(theory_down[mask_pos], t_pts[mask_pos])
area_total_pos = area_up_pos + area_down_pos

# Print results
print("Area in range [-1, 2]:")
print(f"  Up: {area_up_full:.4f}, Down: {area_down_full:.4f}, Total: {area_total_full:.4f}")

print("Area in range [0, 2]:")
print(f"  Up: {area_up_pos:.4f}, Down: {area_down_pos:.4f}, Total: {area_total_pos:.4f}")


In [ ]:
from scipy.integrate import trapezoid as trapz

# Ensure t_pts is a numpy array
t_pts = np.array(t_pts)

# Define masks for the integration ranges
mask_full = (t_pts >= -1) & (t_pts <= 2)
mask_pos = (t_pts >= 0) & (t_pts <= 2)

# Compute areas under the curve for full range [-1, 2]
area_up_full = trapz(theory_up[mask_full], t_pts[mask_full])
area_down_full = trapz(theory_down[mask_full], t_pts[mask_full])
area_total_full = area_up_full + area_down_full

# Compute areas under the curve for positive range [0, 2]
area_up_pos = trapz(theory_up[mask_pos], t_pts[mask_pos])
area_down_pos = trapz(theory_down[mask_pos], t_pts[mask_pos])
area_total_pos = area_up_pos + area_down_pos

# Print results
print("Area in range [-1, 2]:")
print(f"  Up: {area_up_full:.4f}, Down: {area_down_full:.4f}, Total: {area_total_full:.4f}")

print("Area in range [0, 2]:")
print(f"  Up: {area_up_pos:.4f}, Down: {area_down_pos:.4f}, Total: {area_total_pos:.4f}")
